# 무신사 리뷰 스크래핑 및 분석
 

1. 무신사스토어 > 상의 > 셔츠/블라우스 > (후기순 정렬) > 상품 리뷰(스타일 후기) 스크래핑
2. 브랜드 - 상품명 - 리뷰 추출  
3. 각 상품에 따른 리뷰 키워드 추출, 브랜드별 키워드 추출
    - 추출 방법(3): TF-IDF(빈도분석), LSA/LDA, KeyBERT
    - 결과 비교 

In [ ]:
## selenium 설치 
!pip install selenium
!apt-get update
!apt install chromium-chromedriver

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

import time
from tqdm import tqdm

import requests
from bs4 import BeautifulSoup

import pandas as pd

/usr/local/lib/python3.7/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [ ]:
#셀레니움으로 크롬 열기
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')   ## 브라우저가 눈에 안보여도 조작 가능하게
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver', options=chrome_options)

#### 연습 

In [ ]:
cat_num =  '001002'    #셔츠/블라우스
page_num = 1
url = f'https://www.musinsa.com/category/{cat_num}?d_cat_cd={cat_num}&brand=&rate=&page_kind=search&list_kind=small&sort=emt_high&sub_sort=&page={page_num}&display_cnt=90&sale_goods=&group_sale=&kids=N&ex_soldout=&color=&price1=&price2=&exclusive_yn=&shoeSizeOption=&tags=&campaign_id=&timesale_yn=&q=&includeKeywords=&measure='

response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

In [ ]:
boxes = soup.select('div.list-box.box')[0].select('li.li_box')
len(boxes)
#for box in boxes:
#    box.select('div.li_inner')
brand = boxes[3].select('div.li_inner')[0].select('p.item_title')[0].text
prod_name = boxes[3].select('div.li_inner')[0].find('a')['title']
prices = boxes[3].select('div.li_inner')[0].select('p.price')[0].text
url = boxes[3].select('div.li_inner')[0].find('a')['href']

soup.select('div.thumbType_box.box')[0].select('span.totalPagingNum')[0].text

'246'

In [ ]:
url = 'https://store.musinsa.com/app/goods/282220'
# webdriver 실행
driver = webdriver.Chrome('chromedriver', options=chrome_options)
driver.get(url)
driver.find_element(By.CLASS_NAME, "pagingNumber-box.box").find_element(By.CLASS_NAME, 'totalPagingNum').text.split('/')[1].strip()

'652'

## 상품 정보 크롤링(모듈화) 

In [ ]:
## 모듈화

class WebScraping:

    def __init__(self, URL):
        self.url = URL

    def __str__(self):
        return self.url

    def get_max_page_num(self, base_page_num=1):
        
        page_num = base_page_num
        response = requests.get(self.url)
        soup = BeautifulSoup(response.text, 'html.parser')
        total_page_num = soup.select('div.thumbType_box.box')[0].select('span.totalPagingNum')[0].text

        return total_page_num

    def get_product_info(self):

        self.page_num = int(self.get_max_page_num())

        brand_lists = []
        prod_lists = []
        price_lists = []
        url_lists = []

        for page in range(1,self.page_num):   

            page_num = page
            response = requests.get(self.url)
            soup = BeautifulSoup(response.text, 'html.parser')
            boxes = soup.select('div.list-box.box')[0].select('li.li_box')

            for box in boxes:
                brand = box.select('div.li_inner')[0].select('p.item_title')[0].text
                prod_name = box.select('div.li_inner')[0].find('a')['title']
                prices = box.select('div.li_inner')[0].select('p.price')[0].text
                url = box.select('div.li_inner')[0].find('a')['href']

                brand_lists.append(brand)
                prod_lists.append(prod_name)
                price_lists.append(prices)
                url_lists.append(url)

        import pandas as pd
        prod_df = pd.DataFrame({'brand': brand_lists, 'product':prod_lists, 'price':price_lists, 'url':url_lists})
        return prod_df


    ### 메인페이지에서 상품별 주소 추출 


    def get_prod_url(self):
        df = self.get_product_info()
        url_lists = df.url.tolist()
        return url_lists


    def get_max_page_num_for_review(self):
        url_lists = self.get_prod_url()
        max_num_lists = []

        for url in url_lists:
            # webdriver 실행
            driver = webdriver.Chrome('chromedriver', options=chrome_options)
            driver.get(url)
            max_num = driver.find_element(By.CLASS_NAME, "pagingNumber-box.box").find_element(By.CLASS_NAME, 'totalPagingNum').text.split('/')[1].strip()
            max_num_lists.append(max_num)
        
        assert len(url_lists) == len(max_num_lists)

        return max_num_lists

    ### 리뷰 추출

    def get_reviews(self):

        url_lists = self.get_prod_url()
        page_num_lists = self.get_max_page_num_for_review()

        reviews = []

        for url, num in zip(url_lists, page_num_lists):
            
            page_num = num

            # webdriver 실행
            driver = webdriver.Chrome('chromedriver', options=chrome_options)
            driver.get(url)

            # 스타일 리뷰 버튼 (1페이지부터 시작)
            style_review_button = driver.find_element(By.ID, "estimate_style")
            style_review_button.click()  # 클릭하고 
            time.sleep(1)   # 1초 쉬고
                
            review_lists = []


            for page in range(1, page_num+1):

                # 리뷰 리스트(블락) 
                blocks = driver.find_element(By.ID, "reviewListFragment")
                review_blocks = blocks.find_elements(By.CLASS_NAME, "review-list")
                review_lists += [review.find_element(By.CLASS_NAME, 'review-contents__text').text for review in review_blocks]


                # 다음 페이지 넘버 버튼
                pagination = driver.find_element(By.CLASS_NAME, 'wrapper')
                page_cl = pagination.find_elements(By.CSS_SELECTOR, "a.paging-btn.btn")  #
                for pg in page_cl:
                    pg.find_element(By.XPATH, "//a[contains(text(), 'str(page+1)')]").cilck()   # 다음페이지 찾고 클릭

                time.sleep(1)  #로딩하는 시간 1초 쉬고
                
                if page % 5 == 0 : 
                    next_button = driver.find_element(By.CLASS_NAME, "fa.fa-angle-right.paging-btn.btn.next")  #
                    next_button.send_keys(Keys.ENTER)  #클릭! (왜인지 click()이 먹지 않아서 수정)
                    time.sleep(1)  #로딩하는 시간 1초 쉬고       
                
                if page % 10 == 0:
                    print(len(review_lists))
                
            reviews.extend(review_lists)

            review_df = pd.DataFrame({'url':url, 'review': reviews})

            return review_df

In [ ]:
## class 실행 
'''
URL = f'https://www.musinsa.com/category/001004?d_cat_cd=001004&brand=&rate=&page_kind=search&list_kind=small&sort=emt_high&sub_sort=&page={page_num}&display_cnt=90&sale_goods=&group_sale=&kids=N&ex_soldout=&color=&price1=&price2=&exclusive_yn=&shoeSizeOption=&tags=&campaign_id=&timesale_yn=&q=&includeKeywords=&measure='

scrap = WebScraping(URL)
prod_df = scrap.get_product_info()
url_lists = scrap.get_prod_url()
max_num_lists = scrap.get_max_page_num_for_review()
review_df = scrap.get_reviews()

'''

"\nURL = f'https://www.musinsa.com/category/001004?d_cat_cd=001004&brand=&rate=&page_kind=search&list_kind=small&sort=emt_high&sub_sort=&page={page_num}&display_cnt=90&sale_goods=&group_sale=&kids=N&ex_soldout=&color=&price1=&price2=&exclusive_yn=&shoeSizeOption=&tags=&campaign_id=&timesale_yn=&q=&includeKeywords=&measure='\n\nscrap = WebScraping(URL)\nprod_df = scrap.get_product_info()\nurl_lists = scrap.get_prod_url()\nmax_num_lists = scrap.get_max_page_num_for_review()\nreview_df = scrap.get_reviews()\n\n"

## 스크래핑 함수 실행

### 상품 기본정보 추출(메인페이지) 

In [ ]:
## url : 후기순 나열 셔츠/블라우스
url = f'https://www.musinsa.com/category/001002?d_cat_cd=001002&brand=&rate=&page_kind=search&list_kind=small&sort=emt_high&sub_sort=&page={page_num}&display_cnt=90&sale_goods=&group_sale=&kids=N&ex_soldout=&color=&price1=&price2=&exclusive_yn=&shoeSizeOption=&tags=&campaign_id=&timesale_yn=&q=&includeKeywords=&measure='

# 메인페이지에서 max_page_num 추출
def get_max_page_num(url, page_num=1):

    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    total_page_num = soup.select('div.thumbType_box.box')[0].select('span.totalPagingNum')[0].text

    return total_page_num
    

# 상품 정보 추출(requests ver)
def get_product_info(page_number):

    brand_lists = []
    prod_lists = []
    price_lists = []
    url_lists = []

    for page in range(1,page_number):   

        page_num = page
        url = f'https://www.musinsa.com/category/001002?d_cat_cd=001002&brand=&rate=&page_kind=search&list_kind=small&sort=emt_high&sub_sort=&page={page_num}&display_cnt=90&sale_goods=&group_sale=&kids=N&ex_soldout=&color=&price1=&price2=&exclusive_yn=&shoeSizeOption=&tags=&campaign_id=&timesale_yn=&q=&includeKeywords=&measure='

        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        boxes = soup.select('div.list-box.box')[0].select('li.li_box')
        #print(len(boxes))


        for box in boxes:
            brand = box.select('div.li_inner')[0].select('p.item_title')[0].text
            prod_name = box.select('div.li_inner')[0].find('a')['title']
            prices = box.select('div.li_inner')[0].select('p.price')[0].text
            url = box.select('div.li_inner')[0].find('a')['href']

            brand_lists.append(brand)
            prod_lists.append(prod_name)
            price_lists.append(prices)
            url_lists.append(url)
    
    prod_df = pd.DataFrame({'brand': brand_lists, 'product':prod_lists, 'price':price_lists, 'url':url_lists})
    return prod_df


In [ ]:
'''
### 메인페이지에서 상품별 주소 추출(selenium ver)
def get_product_info(url):

    # webdriver 실행
    driver = webdriver.Chrome('chromedriver', options=chrome_options)
    driver.get(url)
        
    brand_lists = []
    #price_lists = []
    url_num_lists = []

    total_page = driver.find_element(By.CLASS_NAME, "thumbType_box.box").find_element(By.XPATH, "//span[@class='totalPagingNum']").text

    for page in range(1, int(total_page)+1):

        # 상품 리스트
        blocks = driver.find_element(By.ID, "goods_list").find_element(By.CLASS_NAME, "list-box.box")
        prod_boxes = blocks.find_elements(By.XPATH, "//li[@class='li_box']")
        for box in prod_boxes:
            url_num = box.get_attribute('data-no')
            prod_info = box.find_element(By.CLASS_NAME, 'li_inner').find_element(By.CLASS_NAME,'article_info')
            brand_name = prod_info.find_element(By.CLASS_NAME, 'item_title').text
            #price = prod_info.find_element(By.CLASS_NAME, 'price').text

            brand_lists.append(brand_name)
            #price_lists.append(price)
            url_num_lists.append(url_num)


        # 다음 페이지 넘버 버튼
        pagination = driver.find_element(By.CLASS_NAME, 'wrapper')
        page_cl = pagination.find_elements(By.CSS_SELECTOR, "a.paging-btn.btn")  #
        for pg in page_cl:
            pg.find_element(By.XPATH, "//a[contains(text(), 'str(page+1)')]").cilck()  
        time.sleep(1)  

        if page % 10 == 0 : 
            next_button = driver.find_element(By.CLASS_NAME, "fa.fa-angle-right.paging-btn.btn.next")  #
            next_button.send_keys(Keys.ENTER) 
            time.sleep(1)    


    import pandas as pd
    review_df = pd.DataFrame({'brand': brand_lists,  'url_num':url_num_lists})
    return review_df
'''

'\n### 메인페이지에서 상품별 주소 추출(selenium ver)\ndef get_product_info(url):\n\n    # webdriver 실행\n    driver = webdriver.Chrome(\'chromedriver\', options=chrome_options)\n    driver.get(url)\n        \n    brand_lists = []\n    #price_lists = []\n    url_num_lists = []\n\n    total_page = driver.find_element(By.CLASS_NAME, "thumbType_box.box").find_element(By.XPATH, "//span[@class=\'totalPagingNum\']").text\n\n    for page in range(1, int(total_page)+1):\n\n        # 상품 리스트\n        blocks = driver.find_element(By.ID, "goods_list").find_element(By.CLASS_NAME, "list-box.box")\n        prod_boxes = blocks.find_elements(By.XPATH, "//li[@class=\'li_box\']")\n        for box in prod_boxes:\n            url_num = box.get_attribute(\'data-no\')\n            prod_info = box.find_element(By.CLASS_NAME, \'li_inner\').find_element(By.CLASS_NAME,\'article_info\')\n            brand_name = prod_info.find_element(By.CLASS_NAME, \'item_title\').text\n            #price = prod_info.find_element(By.CLASS_NAME,

### 리뷰 추출

In [ ]:
#cat = 001004  # 후드티셔츠
#url = f'https://www.musinsa.com/category/001004?d_cat_cd=001004&brand=&rate=&page_kind=search&list_kind=small&sort=emt_high&sub_sort=&page={page_num}&display_cnt=90&sale_goods=&group_sale=&kids=N&ex_soldout=&color=&price1=&price2=&exclusive_yn=&shoeSizeOption=&tags=&campaign_id=&timesale_yn=&q=&includeKeywords=&measure='

prod_df = get_product_info(page_number=5)   # 상품 5페이지
prod_df.shape 

(360, 4)

In [ ]:
prod_df.head(30)

,brand,product,price,url
0,다이아몬드 레이라,플러프 무드 체크 셔츠 S24 네이비,"\n68,000원 51,000원\n",https://store.musinsa.com/app/goods/859956
1,테이크이지,솔리드 옥스포드 오버셔츠(스카이블루),"\n67,000원 53,600원\n",https://store.musinsa.com/app/goods/1336398
2,무신사 스탠다드,베이식 옥스포드 셔츠 [화이트],"\n 24,900원\n ...",https://store.musinsa.com/app/goods/728000
3,쟈니웨스트,Wooltan Check Shirts (Cream),"\n34,000원 32,300원\n",https://store.musinsa.com/app/goods/890749
4,다이아몬드 레이라,빈티지 블랙 도트 스트라이프 셔츠 S22,"\n65,000원 51,000원\n",https://store.musinsa.com/app/goods/826451
5,테이크이지,솔리드 옥스포드 오버셔츠(화이트),"\n67,000원 53,600원\n",https://store.musinsa.com/app/goods/1336290
6,무신사 스탠다드,릴렉스 핏 옥스포드 셔츠 [화이트],"\n 24,900원\n ...",https://store.musinsa.com/app/goods/854337
7,아노블리어,루즈핏 오픈카라 텐셀 린넨 셔츠 [네이비],"\n60,000원 56,400원\n",https://store.musinsa.com/app/goods/1008043
8,라퍼지스토어,시티보이 빅오버 옥스포드 반팔 셔츠_Sky Blue,"\n45,000원 36,000원\n",https://store.musinsa.com/app/goods/1935180
9,파르티멘토,Hair Line Stripe Shirts Sky,"\n39,000원 29,900원\n",https://store.musinsa.com/app/goods/533067


In [ ]:
prods = prod_df.url[:30] # 그 중 30개 url(상품)에 대한 리뷰 추출

In [ ]:
# 상품기본정보 파일 저장
prod_df.to_csv('prod_df.csv', index = False)

In [ ]:
urls = prod_df.url[:30].tolist()

In [ ]:
## 리뷰페이지의 max_page_num 추출
def get_max_page_num_for_review(url_lists):
    max_num_lists = []

    for url in url_lists:
        # webdriver 실행
        driver = webdriver.Chrome('chromedriver', options=chrome_options)
        driver.get(url)
        max_num = driver.find_element(By.CLASS_NAME, "pagingNumber-box.box").find_element(By.CLASS_NAME, 'totalPagingNum').text.split('/')[1].strip()
        max_num_lists.append(max_num)
    
    assert len(url_lists) == len(max_num_lists)

    return max_num_lists

In [ ]:
max_num_lists = get_max_page_num_for_review(urls)

In [ ]:
### 리뷰 추출

def get_reviews(url, page_num):

    # webdriver 실행
    driver = webdriver.Chrome('chromedriver', options=chrome_options)
    driver.get(url)

    # 스타일 리뷰 버튼 (1페이지부터 시작)
    style_review_button = driver.find_element(By.ID, "estimate_style")
    style_review_button.click()  # 클릭하고 
    time.sleep(1)   # 1초 쉬고
        
    review_lists = []


    for page in tqdm(range(1, int(page_num)+1)):

        # 리뷰 리스트(블락) 
        blocks = driver.find_element(By.ID, "reviewListFragment")
        review_blocks = blocks.find_elements(By.CLASS_NAME, "review-list")
        review_lists += [review.find_element(By.CLASS_NAME, 'review-contents__text').text for review in review_blocks]


        # 다음 페이지 넘버 버튼
        pagination = driver.find_element(By.CLASS_NAME, 'wrapper')
        page_cl = pagination.find_elements(By.CSS_SELECTOR, "a.paging-btn.btn")  #
        for pg in page_cl:
            pg.find_element(By.XPATH, "//a[contains(text(), 'str(page+1)')]").cilck()   # 다음페이지 찾고 클릭

        time.sleep(1)  #로딩하는 시간 1초 쉬고
        
        if page % 5 == 0 : 
            next_button = driver.find_element(By.CLASS_NAME, "fa.fa-angle-right.paging-btn.btn.next")  #
            next_button.send_keys(Keys.ENTER)  #클릭! (왜인지 click()이 먹지 않아서 수정)
            time.sleep(1)  #로딩하는 시간 1초 쉬고       
        
        #if page % 10 == 0:
        #    print(len(review_lists))
        
    return review_lists

In [ ]:
## url 따른 상품명 
comb_df = prod_df[['product','url']][:30]
prod_name = []
for i in range(len(urls)):
    prod_name.append(prod_df['product'][i])
print(prod_name)

['플러프 무드 체크 셔츠 S24 네이비', '솔리드 옥스포드 오버셔츠(스카이블루)', '베이식 옥스포드 셔츠 [화이트]', 'Wooltan Check Shirts (Cream)', '빈티지 블랙 도트 스트라이프 셔츠 S22', '솔리드 옥스포드 오버셔츠(화이트)', '릴렉스 핏 옥스포드 셔츠 [화이트]', '루즈핏 오픈카라 텐셀 린넨 셔츠 [네이비]', '시티보이 빅오버 옥스포드 반팔 셔츠_Sky Blue', 'Hair Line Stripe Shirts Sky', '릴렉스 핏 실키 셔츠 [화이트]', '오버핏 옥스포드 셔츠_SPYWC12C02', '루즈핏 오픈카라 텐셀 린넨 셔츠 [오트밀]', '클래식 화이트 셔츠 S40', '릴렉스 핏 실키 셔츠 [블랙]', '릴렉스 핏 실키 셔츠 [다크 네이비]', '릴렉스 핏 실키 셔츠 [그레이쉬 블루]', '[리뉴얼] 옵티멀 베이직 린넨 셔츠-화이트', '플러프 무드 체크 셔츠 S24 브라운', '[유아인 착용 상품] 릴렉스 핏 실키 셔츠 [카키]', '피그먼트 루즈핏 셔츠 베이지 SMLS4058', '루즈핏 오픈카라 텐셀 린넨 셔츠 [블랙]', 'STRIPE HANDSOME DAN SHIRT BLUE', '헤링본 클래식 체크 셔츠 S41', '시티보이 빅오버 옥스포드 반팔 셔츠_White', '[리뉴얼] 오버핏 글림 스트라이프 셔츠-네이비', '옥스포드 베이직 셔츠 화이트', '스티브 보이핏 오버셔츠 WHYAC2225U', '릴렉스 핏 옥스포드 셔츠 [블루]', '[컬러추가]PIN STRIPE OVER SHIRT']


In [ ]:
df_dict = {}

for i in range(len(urls)):
    reviews = get_reviews(urls[i], max_num_lists[i])
    df_dict[prod_name[i]] = pd.DataFrame({'prod_name': prod_name[i], 'review':reviews})
    print(df_dict.keys())

100%|██████████| 242/242 [05:55<00:00,  1.47s/it]


dict_keys(['플러프 무드 체크 셔츠 S24 네이비'])


100%|██████████| 250/250 [06:09<00:00,  1.48s/it]


dict_keys(['플러프 무드 체크 셔츠 S24 네이비', '솔리드 옥스포드 오버셔츠(스카이블루)'])


100%|██████████| 93/93 [02:13<00:00,  1.44s/it]


...

dict_keys(['플러프 무드 체크 셔츠 S24 네이비', '솔리드 옥스포드 오버셔츠(스카이블루)', '베이식 옥스포드 셔츠 [화이트]', 'Wooltan Check Shirts (Cream)', '빈티지 블랙 도트 스트라이프 셔츠 S22', '솔리드 옥스포드 오버셔츠(화이트)', '릴렉스 핏 옥스포드 셔츠 [화이트]', '루즈핏 오픈카라 텐셀 린넨 셔츠 [네이비]', '시티보이 빅오버 옥스포드 반팔 셔츠_Sky Blue', 'Hair Line Stripe Shirts Sky', '릴렉스 핏 실키 셔츠 [화이트]', '오버핏 옥스포드 셔츠_SPYWC12C02', '루즈핏 오픈카라 텐셀 린넨 셔츠 [오트밀]', '클래식 화이트 셔츠 S40', '릴렉스 핏 실키 셔츠 [블랙]', '릴렉스 핏 실키 셔츠 [다크 네이비]', '릴렉스 핏 실키 셔츠 [그레이쉬 블루]', '[리뉴얼] 옵티멀 베이직 린넨 셔츠-화이트', '플러프 무드 체크 셔츠 S24 브라운', '[유아인 착용 상품] 릴렉스 핏 실키 셔츠 [카키]', '피그먼트 루즈핏 셔츠 베이지 SMLS4058', '루즈핏 오픈카라 텐셀 린넨 셔츠 [블랙]', 'STRIPE HANDSOME DAN SHIRT BLUE', '헤링본 클래식 체크 셔츠 S41', '시티보이 빅오버 옥스포드 반팔 셔츠_White', '[리뉴얼] 오버핏 글림 스트라이프 셔츠-네이비', '옥스포드 베이직 셔츠 화이트', '스티브 보이핏 오버셔츠 WHYAC2225U'])


100%|██████████| 88/88 [02:08<00:00,  1.46s/it]


dict_keys(['플러프 무드 체크 셔츠 S24 네이비', '솔리드 옥스포드 오버셔츠(스카이블루)', '베이식 옥스포드 셔츠 [화이트]', 'Wooltan Check Shirts (Cream)', '빈티지 블랙 도트 스트라이프 셔츠 S22', '솔리드 옥스포드 오버셔츠(화이트)', '릴렉스 핏 옥스포드 셔츠 [화이트]', '루즈핏 오픈카라 텐셀 린넨 셔츠 [네이비]', '시티보이 빅오버 옥스포드 반팔 셔츠_Sky Blue', 'Hair Line Stripe Shirts Sky', '릴렉스 핏 실키 셔츠 [화이트]', '오버핏 옥스포드 셔츠_SPYWC12C02', '루즈핏 오픈카라 텐셀 린넨 셔츠 [오트밀]', '클래식 화이트 셔츠 S40', '릴렉스 핏 실키 셔츠 [블랙]', '릴렉스 핏 실키 셔츠 [다크 네이비]', '릴렉스 핏 실키 셔츠 [그레이쉬 블루]', '[리뉴얼] 옵티멀 베이직 린넨 셔츠-화이트', '플러프 무드 체크 셔츠 S24 브라운', '[유아인 착용 상품] 릴렉스 핏 실키 셔츠 [카키]', '피그먼트 루즈핏 셔츠 베이지 SMLS4058', '루즈핏 오픈카라 텐셀 린넨 셔츠 [블랙]', 'STRIPE HANDSOME DAN SHIRT BLUE', '헤링본 클래식 체크 셔츠 S41', '시티보이 빅오버 옥스포드 반팔 셔츠_White', '[리뉴얼] 오버핏 글림 스트라이프 셔츠-네이비', '옥스포드 베이직 셔츠 화이트', '스티브 보이핏 오버셔츠 WHYAC2225U', '릴렉스 핏 옥스포드 셔츠 [블루]'])


100%|██████████| 48/48 [01:08<00:00,  1.43s/it]

dict_keys(['플러프 무드 체크 셔츠 S24 네이비', '솔리드 옥스포드 오버셔츠(스카이블루)', '베이식 옥스포드 셔츠 [화이트]', 'Wooltan Check Shirts (Cream)', '빈티지 블랙 도트 스트라이프 셔츠 S22', '솔리드 옥스포드 오버셔츠(화이트)', '릴렉스 핏 옥스포드 셔츠 [화이트]', '루즈핏 오픈카라 텐셀 린넨 셔츠 [네이비]', '시티보이 빅오버 옥스포드 반팔 셔츠_Sky Blue', 'Hair Line Stripe Shirts Sky', '릴렉스 핏 실키 셔츠 [화이트]', '오버핏 옥스포드 셔츠_SPYWC12C02', '루즈핏 오픈카라 텐셀 린넨 셔츠 [오트밀]', '클래식 화이트 셔츠 S40', '릴렉스 핏 실키 셔츠 [블랙]', '릴렉스 핏 실키 셔츠 [다크 네이비]', '릴렉스 핏 실키 셔츠 [그레이쉬 블루]', '[리뉴얼] 옵티멀 베이직 린넨 셔츠-화이트', '플러프 무드 체크 셔츠 S24 브라운', '[유아인 착용 상품] 릴렉스 핏 실키 셔츠 [카키]', '피그먼트 루즈핏 셔츠 베이지 SMLS4058', '루즈핏 오픈카라 텐셀 린넨 셔츠 [블랙]', 'STRIPE HANDSOME DAN SHIRT BLUE', '헤링본 클래식 체크 셔츠 S41', '시티보이 빅오버 옥스포드 반팔 셔츠_White', '[리뉴얼] 오버핏 글림 스트라이프 셔츠-네이비', '옥스포드 베이직 셔츠 화이트', '스티브 보이핏 오버셔츠 WHYAC2225U', '릴렉스 핏 옥스포드 셔츠 [블루]', '[컬러추가]PIN STRIPE OVER SHIRT'])


In [ ]:
len(df_dict)

30

In [ ]:
prod_name

['플러프 무드 체크 셔츠 S24 네이비',
 '솔리드 옥스포드 오버셔츠(스카이블루)',
 '베이식 옥스포드 셔츠 [화이트]',
 'Wooltan Check Shirts (Cream)',
 '빈티지 블랙 도트 스트라이프 셔츠 S22',
 '솔리드 옥스포드 오버셔츠(화이트)',
 '릴렉스 핏 옥스포드 셔츠 [화이트]',
 '루즈핏 오픈카라 텐셀 린넨 셔츠 [네이비]',
 '시티보이 빅오버 옥스포드 반팔 셔츠_Sky Blue',
 'Hair Line Stripe Shirts Sky',
 '릴렉스 핏 실키 셔츠 [화이트]',
 '오버핏 옥스포드 셔츠_SPYWC12C02',
 '루즈핏 오픈카라 텐셀 린넨 셔츠 [오트밀]',
 '클래식 화이트 셔츠 S40',
 '릴렉스 핏 실키 셔츠 [블랙]',
 '릴렉스 핏 실키 셔츠 [다크 네이비]',
 '릴렉스 핏 실키 셔츠 [그레이쉬 블루]',
 '[리뉴얼] 옵티멀 베이직 린넨 셔츠-화이트',
 '플러프 무드 체크 셔츠 S24 브라운',
 '[유아인 착용 상품] 릴렉스 핏 실키 셔츠 [카키]',
 '피그먼트 루즈핏 셔츠 베이지 SMLS4058',
 '루즈핏 오픈카라 텐셀 린넨 셔츠 [블랙]',
 'STRIPE HANDSOME DAN SHIRT BLUE',
 '헤링본 클래식 체크 셔츠 S41',
 '시티보이 빅오버 옥스포드 반팔 셔츠_White',
 '[리뉴얼] 오버핏 글림 스트라이프 셔츠-네이비',
 '옥스포드 베이직 셔츠 화이트',
 '스티브 보이핏 오버셔츠 WHYAC2225U',
 '릴렉스 핏 옥스포드 셔츠 [블루]',
 '[컬러추가]PIN STRIPE OVER SHIRT']

In [ ]:
# csv 파일로 저장
for i in range(len(urls)):
    df_dict[prod_name[i]].to_csv(f'/content/sample_data/df{prod_name[i]}.csv', index = False)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cd '/content/sample_data'
!pwd

/content


In [ ]:
from glob import glob
import os

df_lists = glob('/content/sample_data/*.csv')[:-5]
#files = [os.path.basename(li) for li in df_lists]
df_lists

file_dict = {}
file_name = []
for i in df_lists:
    key = i[23:-4]
    df = pd.read_csv(i)
    file_dict[key] = df
    file_name.append(key)

In [ ]:
concats = []
for i in range(len(file_name)):
    concats.append(file_dict[file_name[i]])
#print(concats)

In [ ]:
dft = pd.concat(concats)
print(dft.shape)

(40460, 2)


In [ ]:
import os
os.makedirs('/content/drive/MyDrive/musinsa', exist_ok = True)
os.getcwd()
os.chdir('/content/drive/MyDrive')
CUR_DIR = os.getcwd()

dft.to_csv(os.path.join(CUR_DIR,'musinsa/musinsa_review.csv'))

In [ ]:
prod_df.to_csv(os.path.join(CUR_DIR,'musinsa/prod_df.csv'))

#### trouble shooting
* pagination..........................!!
    - 첫번째 seleium!
    - max_page_num 추출을 위한 '.find_element'에서 많은 시간이 투입
    - 아직까지 F12의 오른쪽 마우스키가 작동이 안되 스크래핑 하는게 모두 수동 작업
    - 많은 에러를 경험했으니 좀 더 발전해있길...!!

* [selenium official docs](https://selenium-python.readthedocs.io/locating-elements.html)
* 셀레니움 사용시 특정 element의 클릭이 되지 않을때
    - click() 함수를 사용하게 되면 간단하게 해당 element를 클릭할 수 있는데, 원인 미상으로 가끔씩 클릭이 되지 않는 상황
    ```
    from selenium.webdriver.common.keys import Keys
    driver.find_element_by_id("TOTAL").send_keys(Keys.ENTER)
    ```
    - 위의 방법을 통해도 클릭이 되지 않을 수 있는데, 아래와 같은 방법
    ```
    element = driver.find_element_by_id("TOTAL")
    driver.execute_script("arguments[0].click();", element)
    ```

* 후드 티셔츠 데이터의 리뷰를 모두 추출하려고 했는데 생각보다 데이터 양이 많아서 시간이 너무 오래 걸림
* 일단 목적은 크롤링이 아닌 리뷰 분석이므로 여기까지..!